# Talking Head Anime from a Single Image 2: More Expressive (Manual Poser Tool)

**Instruction**

1. From the main menu, click "Runtime > Change runtime type."
2. Change "Hardware accelerator" to "GPU," and click "Save."
3. Run the four cells below, one by one, in order by clicking the "Play" button to the left of it. Wait for each cell to finish before going to the next one.
4. Scroll down to the end of the last cell, and play with the GUI.

**Constraints on Images**

1. Must be an image of a single humanoid anime character.
2. Must be of size 256x256.
3. The head must be roughly contained in the middle 128x128 middle box.
4. Must have PNG format.
5. Must have an alpha channel.
6. Background pixels must have RGBA=(0,0,0,0). See this link if you do not get clean results.

**Links**

* Github repository: http://github.com/pkhungurn/talking-head-anime-2-demo
* Project writeup: http://pkhungurn.github.io/talking-head-anime-2/

In [ ]:
# Clone the repository
%cd /content
!git clone https://github.com/pkhungurn/talking-head-anime-2-demo.git

/content
Cloning into 'talking-head-anime-2-demo'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 112 (delta 24), reused 106 (delta 21), pack-reused 0
Receiving objects: 100% (112/112), 1.36 MiB | 21.04 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
# CD into the repository directory.
%cd /content/talking-head-anime-2-demo

/content/talking-head-anime-2-demo


In [ ]:
# Download model files
!wget -O data/combiner.pt https://www.dropbox.com/s/at2r3v22xgyoxtk/combiner.pt?dl=0
!wget -O data/eyebrow_decomposer.pt https://www.dropbox.com/s/pbomb5vgens03rk/eyebrow_decomposer.pt?dl=0
!wget -O data/eyebrow_morphing_combiner.pt https://www.dropbox.com/s/yk9m5ok03e0ub1f/eyebrow_morphing_combiner.pt?dl=0
!wget -O data/face_morpher.pt https://www.dropbox.com/s/77sza8qkiwd4qq5/face_morpher.pt?dl=0
!wget -O data/two_algo_face_rotator.pt https://www.dropbox.com/s/ek261g9sspf0cqi/two_algo_face_rotator.pt?dl=0

--2021-03-24 15:38:01--  https://www.dropbox.com/s/at2r3v22xgyoxtk/combiner.pt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/at2r3v22xgyoxtk/combiner.pt [following]
--2021-03-24 15:38:01--  https://www.dropbox.com/s/raw/at2r3v22xgyoxtk/combiner.pt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0ab95c578b2db6c4fc3fdb1c26.dl.dropboxusercontent.com/cd/0/inline/BLRSX_83RhDUhBq3L6G41zO0DGAOBrmd4KbMsnAODpL2MY8snavfjaNE6Rqn9qXwmZaiWnAVEDshEV9TAvxtRrQ1MO0frNtIqdiilt_FBrOS9WWiFj5yY9G6TmgSkF6U0UYe5pjGqgnTUbpjLGPQ6LJo/file# [following]
--2021-03-24 15:38:02--  https://uc0ab95c578b2db6c4fc3fdb1c26.dl.dropboxusercontent.com/cd/0/inline/BLRSX_83RhDUhBq3L6G41zO0DGAOBrmd4KbMsnAODpL2MY8snavfjaNE6Rqn9qXwmZaiWnAVEDshEV9TAvxtRrQ1

In [ ]:
import torch

FRAME_RATE = 30.0
DEVICE_NAME = 'cuda'
device = torch.device(DEVICE_NAME)

import PIL.Image
import io
from io import StringIO, BytesIO
import IPython.display
import numpy
import ipywidgets
from tha2.util import extract_pytorch_image_from_filelike, convert_output_image_from_torch_to_numpy
import tha2.poser.modes.mode_20
import time
import threading

last_torch_input_image = None
torch_input_image = None

def show_pytorch_image(pytorch_image, output_widget=None):
    output_image = pytorch_image.detach().cpu()
    numpy_image = convert_output_image_from_torch_to_numpy(output_image)
    pil_image = PIL.Image.fromarray(numpy_image, mode='RGBA')
    IPython.display.display(pil_image)

input_image_widget = ipywidgets.Output(
    layout={
        'border': '1px solid black',
        'width': '256px',
        'height': '256px'
    })

upload_input_image_button = ipywidgets.FileUpload(
    accept='.png',
    multiple=False,
    layout={
        'width': '256px'
    }
)

output_image_widget = ipywidgets.Output(
    layout={
        'border': '1px solid black',
        'width': '256px',
        'height': '256px'
    }
)

eyebrow_dropdown = ipywidgets.Dropdown(
    options=["troubled", "angry", "lowered", "raised", "happy", "serious"],
    value="troubled",
    description="Eyebrow:",    
)
eyebrow_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Left:",
    readout=True,
    readout_format=".2f"
)
eyebrow_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Right:",
    readout=True,
    readout_format=".2f"
)

eye_dropdown = ipywidgets.Dropdown(
    options=["wink", "happy_wink", "surprised", "relaxed", "unimpressed", "raised_lower_eyelid"],
    value="wink",
    description="Eye:",    
)
eye_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Left:",
    readout=True,
    readout_format=".2f"
)
eye_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Right:",
    readout=True,
    readout_format=".2f"
)

mouth_dropdown = ipywidgets.Dropdown(
    options=["aaa", "iii", "uuu", "eee", "ooo", "delta", "lowered_corner", "raised_corner", "smirk"],
    value="aaa",
    description="Mouth:",    
)
mouth_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Value:",
    readout=True,
    readout_format=".2f"
)
mouth_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description=" ",
    readout=True,
    readout_format=".2f",
    disabled=True,
)

def update_mouth_sliders(change):
    if mouth_dropdown.value == "lowered_corner" or mouth_dropdown.value == "raised_corner":
        mouth_left_slider.description = "Left:"
        mouth_right_slider.description = "Right:"
        mouth_right_slider.disabled = False
    else:
        mouth_left_slider.description = "Value:"
        mouth_right_slider.description = " "
        mouth_right_slider.disabled = True

mouth_dropdown.observe(update_mouth_sliders, names='value')

iris_small_left_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Left:",
    readout=True,
    readout_format=".2f"
)
iris_small_right_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Right:",
    readout=True,
    readout_format=".2f",    
)
iris_rotation_x_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="X-axis:",
    readout=True,
    readout_format=".2f"
)
iris_rotation_y_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Y-axis:",
    readout=True,
    readout_format=".2f",    
)

head_x_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="X-axis:",
    readout=True,
    readout_format=".2f"
)
head_y_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Y-axis:",
    readout=True,
    readout_format=".2f",    
)
neck_z_slider = ipywidgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="Z-axis:",
    readout=True,
    readout_format=".2f",    
)


control_panel = ipywidgets.VBox([
    eyebrow_dropdown,
    eyebrow_left_slider,
    eyebrow_right_slider,
    ipywidgets.HTML(value="<hr>"),
    eye_dropdown,
    eye_left_slider,
    eye_right_slider,
    ipywidgets.HTML(value="<hr>"),
    mouth_dropdown,
    mouth_left_slider,
    mouth_right_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Iris Shrinkage</b></center>"),
    iris_small_left_slider,
    iris_small_right_slider,
    ipywidgets.HTML(value="<center><b>Iris Rotation</b></center>"),
    iris_rotation_x_slider,
    iris_rotation_y_slider,
    ipywidgets.HTML(value="<hr>"),
    ipywidgets.HTML(value="<center><b>Head Rotation</b></center>"),
    head_x_slider,
    head_y_slider,
    neck_z_slider,
])

controls = ipywidgets.HBox([
    ipywidgets.VBox([
        input_image_widget, 
        upload_input_image_button
    ]),
    control_panel,
    ipywidgets.HTML(value="&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"),
    output_image_widget,
])

poser = tha2.poser.modes.mode_20.create_poser(device)
pose_parameters = tha2.poser.modes.mode_20.get_pose_parameters()
pose_size = poser.get_num_parameters()
last_pose = torch.zeros(1, pose_size).to(device)

iris_small_left_index = pose_parameters.get_parameter_index("iris_small_left")
iris_small_right_index = pose_parameters.get_parameter_index("iris_small_right")
iris_rotation_x_index = pose_parameters.get_parameter_index("iris_rotation_x")
iris_rotation_y_index = pose_parameters.get_parameter_index("iris_rotation_y")
head_x_index = pose_parameters.get_parameter_index("head_x")
head_y_index = pose_parameters.get_parameter_index("head_y")
neck_z_index = pose_parameters.get_parameter_index("neck_z")

def get_pose():
    pose = torch.zeros(1, pose_size)
    
    eyebrow_name = f"eyebrow_{eyebrow_dropdown.value}"
    eyebrow_left_index = pose_parameters.get_parameter_index(f"{eyebrow_name}_left")
    eyebrow_right_index = pose_parameters.get_parameter_index(f"{eyebrow_name}_right")
    pose[0, eyebrow_left_index] = eyebrow_left_slider.value
    pose[0, eyebrow_right_index] = eyebrow_right_slider.value
    
    eye_name = f"eye_{eye_dropdown.value}"
    eye_left_index = pose_parameters.get_parameter_index(f"{eye_name}_left")
    eye_right_index = pose_parameters.get_parameter_index(f"{eye_name}_right")
    pose[0, eye_left_index] = eye_left_slider.value
    pose[0, eye_right_index] = eye_right_slider.value
    
    mouth_name = f"mouth_{mouth_dropdown.value}"
    if mouth_name == "mouth_lowered_cornered" or mouth_name == "mouth_raised_corner":
        mouth_left_index = pose_parameters.get_parameter_index(f"{mouth_name}_left")
        mouth_right_index = pose_parameters.get_parameter_index(f"{mouth_name}_right")
        pose[0, mouth_left_index] = mouth_left_slider.value
        pose[0, mouth_right_index] = mouth_right_slider.value
    else:
        mouth_index = pose_parameters.get_parameter_index(mouth_name)
        pose[0, mouth_index] = mouth_left_slider.value
    
    pose[0, iris_small_left_index] = iris_small_left_slider.value
    pose[0, iris_small_right_index] = iris_small_right_slider.value
    pose[0, iris_rotation_x_index] = iris_rotation_x_slider.value
    pose[0, iris_rotation_y_index] = iris_rotation_y_slider.value
    pose[0, head_x_index] = head_x_slider.value
    pose[0, head_y_index] = head_y_slider.value
    pose[0, neck_z_index] = neck_z_slider.value
    
    return pose.to(device)

display(controls)

def update(change):
    global last_pose
    global last_torch_input_image
        
    if torch_input_image is None:
        return
        
    needs_update = False
    if last_torch_input_image is None:
        needs_update = True        
    else:
        if (torch_input_image - last_torch_input_image).abs().max().item() > 0:
            needs_update = True         
            
    pose = get_pose()
    if (pose - last_pose).abs().max().item() > 0:
        needs_update = True
    
    if not needs_update:
        return
   
    output_image = poser.pose(torch_input_image, pose)[0]
    with output_image_widget:
        output_image_widget.clear_output(wait=True)
        show_pytorch_image(output_image, output_image_widget)  
        
    last_torch_input_image = torch_input_image
    last_pose = pose
        
def upload_image(change):
    global torch_input_image
    for name, file_info in upload_input_image_button.value.items():
        torch_input_image = extract_pytorch_image_from_filelike(io.BytesIO(file_info['content'])).to(device)
    if torch_input_image is not None:
        c,h,w = torch_input_image.shape
        if h != 256 or w != 256:
            with input_image_widget:
                input_image_widget.clear_output(wait=True)
                display(ipywidgets.HTML("Image must be 256x256 in size!!!"))
            torch_input_image = None
        if c != 4:
            with input_image_widget:
                input_image_widget.clear_output(wait=True)
                display(ipywidgets.HTML("Image must have an alpha channel!!!"))                
            torch_input_image = None
        if torch_input_image is not None:
            with input_image_widget:
                input_image_widget.clear_output(wait=True)
                show_pytorch_image(torch_input_image, input_image_widget)
        update(None)
        
upload_input_image_button.observe(upload_image, names='value')
eyebrow_dropdown.observe(update, 'value')
eyebrow_left_slider.observe(update, 'value')
eyebrow_right_slider.observe(update, 'value')
eye_dropdown.observe(update, 'value')
eye_left_slider.observe(update, 'value')
eye_right_slider.observe(update, 'value')
mouth_dropdown.observe(update, 'value')
mouth_left_slider.observe(update, 'value')
mouth_right_slider.observe(update, 'value')
iris_small_left_slider.observe(update, 'value')
iris_small_right_slider.observe(update, 'value')
iris_rotation_x_slider.observe(update, 'value')
iris_rotation_y_slider.observe(update, 'value')
head_x_slider.observe(update, 'value')
head_y_slider.observe(update, 'value')
neck_z_slider.observe(update, 'value')

In [1]:
!explorer .

In [2]:
print('Git clone project and install requirements...')
!git clone https://github.com/yzhou359/MakeItTalk
%cd MakeItTalk/
!export PYTHONPATH=/content/MakeItTalk:$PYTHONPATH
!pip install -r requir3133/ ements.txt &> /dev/null
!pip install tensorboardX &> /dev/null
!mkdir examples/dump
!mkdir examples/ckpt
!pip install gdown &> /dev/null
print('Done!')
print('Download pre-trained models...')
#!gdown -O examples/ckpt/ckpt_autovc.pth https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
#!gdown -O examples/ckpt/ckpt_content_branch.pth https://drive.google.com/uc?id=1r3bfEvTVl6pCNw5xwUhEglwDHjWtAqQp
#!gdown -O examples/ckpt/ckpt_speaker_branch.pth https://drive.google.com/uc?id=1rV0jkyDqPW-aDJcj7xSO6Zt1zSXqn1mu
#!gdown -O examples/ckpt/ckpt_116_i2i_comb.pth https://drive.google.com/uc?id=1i2LJXKp-yWKIEEgJ7C6cE3_2NirfY_0a
#!gdown -O examples/dump/emb.pickle https://drive.google.com/uc?id=18-0CYl5E6ungS3H4rRSHjfYvvm-WwjTI
print('Done!')

Git clone project and install requirements...
[WinError 2] The system cannot find the file specified: 'MakeItTalk/'
C:\Users\r2q2\opt\Saati_to_commit\notebooks


The syntax of the command is incorrect.
'export' is not recognized as an internal or external command,
operable program or batch file.
The syntax of the command is incorrect.
The syntax of the command is incorrect.
The syntax of the command is incorrect.
The syntax of the command is incorrect.


Done!
Download pre-trained models...
Done!


The syntax of the command is incorrect.


In [ ]:
import ipywidgets as widgets
import glob
import matplotlib.pyplot as plt
print("Choose the image name to animate: (saved in folder 'examples/')")
img_list = glob.glob1('examples', '*.jpg')
img_list.sort()
img_list = [item.split('.')[0] for item in img_list]
default_head_name = widgets.Dropdown(options=img_list, value='paint_boy')
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        plt.imshow(plt.imread('examples/{}.jpg'.format(default_head_name.value)))
        plt.axis('off')
        plt.show()
default_head_name.observe(on_change)
display(default_head_name)
plt.imshow(plt.imread('examples/{}.jpg'.format(default_head_name.value)))
plt.axis('off')
plt.show()

In [1]:
import sys
sys.path.append("thirdparty/AdaptiveWingLoss")
import os, glob
import numpy as np
import cv2
import argparse
from src.approaches.train_image_translation import Image_translation_block
import torch
import pickle
import face_alignment
from src.autovc.AutoVC_mel_Convertor_retrain_version import AutoVC_mel_Convertor
import shutil
import time
import util.utils as util
from scipy.signal import savgol_filter
from src.approaches.train_audio2landmark import Audio2landmark_model

sys.stdout = open(os.devnull, 'a')

parser = argparse.ArgumentParser()
parser.add_argument('--jpg', type=str, default='{}.jpg'.format(default_head_name.value))
parser.add_argument('--close_input_face_mouth', default=CLOSE_INPUT_FACE_MOUTH, action='store_true')
parser.add_argument('--load_AUTOVC_name', type=str, default='../src/MakeItTalk/examples/ckpt/ckpt_autovc.pth')
parser.add_argument('--load_a2l_G_name', type=str, default='../src/MaketItTalk/examples/ckpt/ckpt_speaker_branch.pth')
parser.add_argument('--load_a2l_C_name', type=str, default='../src/MakeItTalk/examples/ckpt/ckpt_content_branch.pth') #ckpt_audio2landmark_c.pth')
parser.add_argument('--load_G_name', type=str, default='../src/MakeItTalk/examples/ckpt/ckpt_116_i2i_comb.pth') #ckpt_image2image.pth') #ckpt_i2i_finetune_150.pth') #c
parser.add_argument('--amp_lip_x', type=float, default=AMP_LIP_SHAPE_X)
parser.add_argument('--amp_lip_y', type=float, default=AMP_LIP_SHAPE_Y)
parser.add_argument('--amp_pos', type=float, default=AMP_HEAD_POSE_MOTION)
parser.add_argument('--reuse_train_emb_list', type=str, nargs='+', default=[]) #  ['iWeklsXc0H8']) #['45hn7-LXDX8']) #['E_kmpT-EfOg']) #'iWeklsXc0H8', '29k8RtSUjE0', '45hn7-LXDX8',
parser.add_argument('--add_audio_in', default=False, action='store_true')
parser.add_argument('--comb_fan_awing', default=False, action='store_true')
parser.add_argument('--output_folder', type=str, default='examples')
parser.add_argument('--test_end2end', default=True, action='store_true')
parser.add_argument('--dump_dir', type=str, default='', help='')
parser.add_argument('--pos_dim', default=7, type=int)
parser.add_argument('--use_prior_net', default=True, action='store_true')
parser.add_argument('--transformer_d_model', default=32, type=int)
parser.add_argument('--transformer_N', default=2, type=int)
parser.add_argument('--transformer_heads', default=2, type=int)
parser.add_argument('--spk_emb_enc_size', default=16, type=int)
parser.add_argument('--init_content_encoder', type=str, default='')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--reg_lr', type=float, default=1e-6, help='weight decay')
parser.add_argument('--write', default=False, action='store_true')
parser.add_argument('--segment_batch_size', type=int, default=1, help='batch size')
parser.add_argument('--emb_coef', default=3.0, type=float)
parser.add_argument('--lambda_laplacian_smooth_loss', default=1.0, type=float)
parser.add_argument('--use_11spk_only', default=False, action='store_true')
parser.add_argument('-f')
opt_parser = parser.parse_args()

img = cv2.imread('examples/' + opt_parser.jpg)
predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, device='cpu', flip_input=True)
shapes = predictor.get_landmarks(img)
if (not shapes or len(shapes) != 1):
    print('Cannot detect face landmarks. Exit.')
    exit(-1)
shape_3d = shapes[0]
if(opt_parser.close_input_face_mouth):
    util.close_input_face_mouth(shape_3d)
shape_3d[48:, 0] = (shape_3d[48:, 0] - np.mean(shape_3d[48:, 0])) * LIP_WIDTH_ADJUST + np.mean(shape_3d[48:, 0]) # wider lips
shape_3d[49:54, 1] -= UPPER_LIP_ADJUST           # thinner upper lip
shape_3d[55:60, 1] += LOWER_LIP_ADJUST           # thinner lower lip
shape_3d[[37,38,43,44], 1] -=2.    # larger eyes
shape_3d[[40,41,46,47], 1] +=2.    # larger eyes
shape_3d, scale, shift = util.norm_input_face(shape_3d)

print("Loaded Image...", file=sys.stderr)

au_data = []
au_emb = []
ains = glob.glob1('examples', '*.wav')
ains = [item for item in ains if item is not 'tmp.wav']
ains.sort()
for ain in ains:
    os.system('ffmpeg -y -loglevel error -i examples/{} -ar 16000 examples/tmp.wav'.format(ain))
    shutil.copyfile('examples/tmp.wav', 'examples/{}'.format(ain))

    # au embedding
    from thirdparty.resemblyer_util.speaker_emb import get_spk_emb
    me, ae = get_spk_emb('examples/{}'.format(ain))
    au_emb.append(me.reshape(-1))

    print('Processing audio file', ain)
    c = AutoVC_mel_Convertor('examples')

    au_data_i = c.convert_single_wav_to_autovc_input(audio_filename=os.path.join('examples', ain),
           autovc_model_path=opt_parser.load_AUTOVC_name)
    au_data += au_data_i
if(os.path.isfile('examples/tmp.wav')):
    os.remove('examples/tmp.wav')

print("Loaded audio...", file=sys.stderr)

# landmark fake placeholder
fl_data = []
rot_tran, rot_quat, anchor_t_shape = [], [], []
for au, info in au_data:
    au_length = au.shape[0]
    fl = np.zeros(shape=(au_length, 68 * 3))
    fl_data.append((fl, info))
    rot_tran.append(np.zeros(shape=(au_length, 3, 4)))
    rot_quat.append(np.zeros(shape=(au_length, 4)))
    anchor_t_shape.append(np.zeros(shape=(au_length, 68 * 3)))

if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_fl.pickle'))
if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))
if(os.path.exists(os.path.join('examples', 'dump', 'random_val_au.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_au.pickle'))
if (os.path.exists(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))

with open(os.path.join('examples', 'dump', 'random_val_fl.pickle'), 'wb') as fp:
    pickle.dump(fl_data, fp)
with open(os.path.join('examples', 'dump', 'random_val_au.pickle'), 'wb') as fp:
    pickle.dump(au_data, fp)
with open(os.path.join('examples', 'dump', 'random_val_gaze.pickle'), 'wb') as fp:
    gaze = {'rot_trans':rot_tran, 'rot_quat':rot_quat, 'anchor_t_shape':anchor_t_shape}
    pickle.dump(gaze, fp)

model = Audio2landmark_model(opt_parser, jpg_shape=shape_3d)
if(len(opt_parser.reuse_train_emb_list) == 0):
    model.test(au_emb=au_emb)
else:
    model.test(au_emb=None)

print("Audio->Landmark...", file=sys.stderr)

fls = glob.glob1('examples', 'pred_fls_*.txt')
fls.sort()

for i in range(0,len(fls)):
    fl = np.loadtxt(os.path.join('examples', fls[i])).reshape((-1, 68,3))
    fl[:, :, 0:2] = -fl[:, :, 0:2]
    fl[:, :, 0:2] = fl[:, :, 0:2] / scale - shift

    if (ADD_NAIVE_EYE):
        fl = util.add_naive_eye(fl)

    # additional smooth
    fl = fl.reshape((-1, 204))
    fl[:, :48 * 3] = savgol_filter(fl[:, :48 * 3], 15, 3, axis=0)
    fl[:, 48*3:] = savgol_filter(fl[:, 48*3:], 5, 3, axis=0)
    fl = fl.reshape((-1, 68, 3))

    ''' STEP 6: Imag2image translation '''
    model = Image_translation_block(opt_parser, single_test=True)
    with torch.no_grad():
        model.single_test(jpg=img, fls=fl, filename=fls[i], prefix=opt_parser.jpg.split('.')[0])
        print('finish image2image gen')
    os.remove(os.path.join('examples', fls[i]))

    print("{} / {}: Landmark->Face...".format(i+1, len(fls)), file=sys.stderr)
print("Done!", file=sys.stderr)

<>:74: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:74: SyntaxWarning: "is not" with a literal. Did you mean "!="?


ModuleNotFoundError: No module named 'src'